In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

# pour accéder aux fichiers
import os
# faire des opérations sur images
import cv2
# pour afficher images
import PIL

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# afficher predictions
from sklearn.metrics import classification_report

In [ ]:
import pathlib
import zipfile
import tensorflow as tf

urls = ["https://github.com/cai-i/image_classification/raw/train_test/resized_images.zip"] 
for url in urls :
  data_dir = tf.keras.utils.get_file(origin=url, extract=False)

  with zipfile.ZipFile(data_dir, 'r') as zip_ref:
    zip_ref.extractall('/content/datasets')

In [ ]:
data_dir = pathlib.Path('/content/datasets/resized_images')
train_dir = pathlib.Path('/content/datasets/resized_images/train')
val_dir = pathlib.Path('/content/datasets/resized_images/val')
test_dir = pathlib.Path('/content/datasets/resized_images/test')

In [ ]:
class_names = ['abeille', 'guepe', 'frelon']
class_names_label = { class_name:i for i, class_name in enumerate(class_names)}

print(class_names_label)

In [ ]:
def load_data():

  dir = '/content/datasets/resized_images/'
  datasets = ['train', 'val', 'test']

  output = []

  for dataset in datasets:
    path = os.path.join(dir, dataset)
    images=[]
    labels=[]

    for category in os.listdir(path):
      label= class_names_label[category]
      cat_path = os.path.join(path, category)

      for img in os.listdir(cat_path):
        img_path = os.path.join(cat_path, img)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # append to lists
        images.append(image)
        labels.append(label)
    
    images = np.array(images, dtype='float32')
    labels = np.array(labels, dtype = 'int32')

    output.append((images, labels))

  return output

In [ ]:
(train_imgs, train_labels), (val_imgs, val_labels), (test_imgs, test_labels) = load_data()

In [ ]:
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.models import Model

In [ ]:
model_vgg = VGG16(weights='imagenet', include_top=False)
model_vgg = Model(inputs=model_vgg.inputs, outputs=model_vgg.layers[-5].output)

In [ ]:
train_features = model_vgg.predict(train_imgs)
val_features = model_vgg.predict(val_imgs)
test_features = model_vgg.predict(test_imgs)

In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Activation

model = VGG16(weights='imagenet', include_top=False)

input_shape = model.layers[-4].get_input_shape_at(0)
layer_input = Input(shape=(9,9,512))

x = layer_input
for layer in model.layers[-4::1]:
  x = layer(x)

x = Conv2D(64, (3,3), activation='relu')(x)
x = MaxPooling2D(pool_size = (2,2))(x)
x = Flatten()(x)
x = Dense(100, activation = 'relu')(x)
x = Dense(6, activation='softmax')(x)

# create the model
model = Model(layer_input, x)

In [ ]:
model.compile(
    optimizer='adam',
    loss= 'sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_features,
    train_labels,
    batch_size= 128,
    epochs=20,
    validation_data= (val_features,val_labels)
)

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

In [ ]:
model.summary()

In [ ]:
test_loss = model.evaluate(test_imgs, test_labels)

In [ ]:
predictions = model.predict(test_imgs)
pred_labels = np.argmax(predictions, axis=1)

print(classification_report(test_labels, pred_labels))